# Exercício 3 – Back Marketing

- https://www.kaggle.com/datasets/henriqueyamahata/bank-marketing 

Tarefa: 
- Prever se o cliente adere ao produto bancário 
- Codificar variáveis com OneHotEncoder 
- Comparar desempenho com regressão logística e árvore de decisão

In [103]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [104]:
import zipfile

zip_path = 'data/bank-additional-full.csv.zip'
file_name = 'bank-additional-full.csv'

with zipfile.ZipFile(zip_path) as z:
    info = z.getinfo(file_name)
    print(f"Tamanho do ficheiro '{file_name}': {info.file_size//1024} kbytes")

    with z.open(file_name) as f:
        df = pd.read_csv(f, sep=';')

Tamanho do ficheiro 'bank-additional-full.csv': 5698 kbytes


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [106]:
colunas_originais = list(df.columns)
print('Colunas originais:', colunas_originais)

Colunas originais: ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y']


In [107]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


# Renomear algumas colunas

In [109]:
# fix names
df.rename(
    columns=
        {
            'age': 'idade',
            'job': 'profissao',
            'marital': 'estado_civil',
            'education': 'escolaridade',
            'y': 'subscrito'

        }, 
        inplace=True)
colunas_novas = list(df.columns)

print(f"Colunas originais: {colunas_originais}")
print(f"Colunas novas: {colunas_novas}")

Colunas originais: ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y']
Colunas novas: ['idade', 'profissao', 'estado_civil', 'escolaridade', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'subscrito']


# Separar features e target

In [110]:
#X = df[['idade', 'profissao']] # variáveis independentes
categorical_features = ['profissao', 'estado_civil', 'escolaridade']
numerical_features = ['idade']

X = df[categorical_features + numerical_features]
y = df['subscrito']    # variável dependente

In [111]:
df['subscrito'].value_counts()

subscrito
no     36548
yes     4640
Name: count, dtype: int64

# Definir o ColumnTransformer com OneHotEncoder 

In [112]:
preprocessador = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_features), 
    ],
    remainder='passthrough'
)

# Criar um pipeline com regressão logística

In [113]:
model = Pipeline(
    steps=[
        ('preprocessamento', preprocessador),
        ('classificador', LogisticRegression(max_iter=1000, random_state=42))
    ]
)

# max_iter =1000 to ensure convergence
# senão há este erro: STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

# Treinar o modelo

In [114]:
model.fit(X, y)

,steps,"[('preprocessamento', ...), ('classificador', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


# Visualização das variáveis codificadas

In [115]:
# ver os nomes das colunas após transformação
nomes_colunas = model.named_steps['preprocessamento'].get_feature_names_out()
print(nomes_colunas)

['cat__profissao_blue-collar' 'cat__profissao_entrepreneur'
 'cat__profissao_housemaid' 'cat__profissao_management'
 'cat__profissao_retired' 'cat__profissao_self-employed'
 'cat__profissao_services' 'cat__profissao_student'
 'cat__profissao_technician' 'cat__profissao_unemployed'
 'cat__profissao_unknown' 'cat__estado_civil_married'
 'cat__estado_civil_single' 'cat__estado_civil_unknown'
 'cat__escolaridade_basic.6y' 'cat__escolaridade_basic.9y'
 'cat__escolaridade_high.school' 'cat__escolaridade_illiterate'
 'cat__escolaridade_professional.course'
 'cat__escolaridade_university.degree' 'cat__escolaridade_unknown'
 'remainder__idade']


# Simulação

In [91]:
df['estado_civil'].unique()

array(['married', 'single', 'divorced', 'unknown'], dtype=object)

Hiper simulação

In [121]:
import itertools

# Listas de valores possíveis
idades = list(range(18, 67))
profissoes = df['profissao'].unique()
estados_civil = df['estado_civil'].unique()
escolaridades = df['escolaridade'].unique()

# Gerar todas as combinações possíveis
combinacoes = itertools.product(idades, profissoes, estados_civil, escolaridades)

# Criar DataFrame de simulação
simulacoes = pd.DataFrame(combinacoes, columns=['idade', 'profissao', 'estado_civil', 'escolaridade'])

# Fazer previsões
simulacoes['subscricao_prevista'] = model.predict(simulacoes)
simulacoes['probabilidade'] = model.predict_proba(simulacoes)[:, 1]

# Ordenar pelo valor da probabilidade (decrescente)
simulacoes_ordenado = simulacoes.sort_values(by='probabilidade', ascending=False)

# Mostrar as 10 combinações com maior probabilidade
top10 = simulacoes_ordenado.head(10)

for idx, row in top10.iterrows():
    print(
        f"Idade: {row['idade']:2} | "
        f"Profissão: {row['profissao']:15} | "
        f"Estado Civil: {row['estado_civil']:10} | "
        f"Escolaridade: {row['escolaridade']:20} | "
        f"Subscrição prevista: {row['subscricao_prevista']} | "
        f"Probabilidade: {row['probabilidade']:.2%}"
    )

Idade: 66 | Profissão: student         | Estado Civil: single     | Escolaridade: university.degree    | Subscrição prevista: no | Probabilidade: 45.13%
Idade: 65 | Profissão: student         | Estado Civil: single     | Escolaridade: university.degree    | Subscrição prevista: no | Probabilidade: 44.87%
Idade: 64 | Profissão: student         | Estado Civil: single     | Escolaridade: university.degree    | Subscrição prevista: no | Probabilidade: 44.62%
Idade: 66 | Profissão: student         | Estado Civil: single     | Escolaridade: unknown              | Subscrição prevista: no | Probabilidade: 44.49%
Idade: 63 | Profissão: student         | Estado Civil: single     | Escolaridade: university.degree    | Subscrição prevista: no | Probabilidade: 44.36%
Idade: 65 | Profissão: student         | Estado Civil: single     | Escolaridade: unknown              | Subscrição prevista: no | Probabilidade: 44.23%
Idade: 62 | Profissão: student         | Estado Civil: single     | Escolaridade: 

In [122]:
print(simulacoes['probabilidade'].describe())

count    18816.000000
mean         0.131597
std          0.069799
min          0.044570
25%          0.086735
50%          0.107705
75%          0.144322
max          0.451289
Name: probabilidade, dtype: float64


# Análise do ficheiro e dos resultados

## 1. Leitura e preparação dos dados
- O ficheiro lê corretamente o CSV a partir do ZIP.
- As colunas são renomeadas para português, facilitando a leitura.
- As features escolhidas para o modelo são razoáveis: `idade`, `profissao`, `estado_civil`, `escolaridade`.

## 2. Pré-processamento
- É usado OneHotEncoder para as variáveis categóricas, o que é adequado.
- A coluna `idade` é passada diretamente (pode ser melhorada com normalização, mas não é obrigatório para regressão logística).

## 3. Treino do modelo
- O pipeline com LogisticRegression está correto.
- O modelo é treinado com todas as linhas do dataset (não há divisão treino/teste, o que pode levar a overfitting, mas para simulação exploratória é aceitável).

## 4. Simulação
- São geradas todas as combinações possíveis de idade, profissão, estado civil e escolaridade.
- Para cada combinação, é feita a previsão da subscrição e da probabilidade.

## 5. Resultados
- As simulações são ordenadas pela maior probabilidade de subscrição e são mostradas as 10 melhores.

---

## Faz sentido os resultados?

- **Se as probabilidades são todas muito baixas:**  
  O modelo pode estar a prever "não" para quase todos os perfis. Isto é comum se a variável target está muito desbalanceada (muito mais "no" do que "yes").

- **Se as profissões, idades ou escolaridades com maior probabilidade fazem sentido:**  
  Por exemplo, se pessoas com certas profissões, idades médias e escolaridade superior têm maior probabilidade, isso pode refletir padrões reais do dataset.

- **Se vês sempre as mesmas profissões/estados civis/escolaridades no top 10:**  
  Pode indicar que o modelo está a dar demasiado peso a uma feature ou que o dataset tem poucos exemplos positivos.

---